In [1]:
from keras.models import load_model
import pandas as pd
from pathlib import Path
import pickle


class ModelProvider(object):

    def __init__(self, model='trained_model.h5', scaler='scaler.pkl'):
        """
        Constructor
        """
        self._scaler_path = self._path_to(scaler)
        self._model_path = self._path_to(model)
        self._model = load_model(self._model_path)
        self.scaler = self._load_scaler(self._scaler_path)
        self._a = self.scaler.min_[0]
        self._b = self.scaler.scale_[0]

    def _load_scaler(self, scaler):
        """
        Loader
        """
        with open(scaler, "rb") as f:
            scaler = pickle.load(f)
        return scaler

    def _rescale(self, Y):
        """
        Aux. method for rescaling results
        """
        return (Y - self._a)/self._b

    def _scale(self, df):
        """
        Scaling
        """
        scaling_training_df = self.scaler.transform(df)
        return pd.DataFrame(scaling_training_df, columns=df.columns.values)

    def _path_to(self, path):
        """
        Aux. method for path to
        """
        path_object = Path(path)
        if path_object.is_file():
            return path
        else:
            return ValueError("You should provide a proper path")

    def predict(self, df):
        """
        Predicts
        """
        scaled_df = self._scale(df)
        X = self._preprocessing(scaled_df)
        raw_prediction = self._model.predict(X)
        return self._rescale(raw_prediction[0][0])

    def _preprocessing(self, scaled_df):
        """
        Preprocessing
        """
        return scaled_df.drop('price', axis=1).values

    def evaluate_error(self, X_test, Y_test, verbose=0):
        """
        Evaluating
        """
        return self._model.evaluate(X_test, Y_test, verbose=verbose)


Using TensorFlow backend.


In [2]:
    # Input data
    df = pd.read_csv("new_house.csv")

    # create and instance of the model
    model = ModelProvider(model='trained_model.h5', scaler='scaler.pkl')

    # Prediction
    prediction = model.predict(df)

    print("Current value - € {}".format(df.iloc[0][0]))
    print("Prediction - € {}".format(prediction))

Current value - € 207949.0
Prediction - € 207995.49307668206
